In [6]:
pip install azure-cognitiveservices-vision-customvision

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install gradio_modal

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from gradio_modal import Modal

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Initialize the prediction client
ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"
 
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 음성 멈춤 기능을 위한 전역 변수
is_speaking = False

# 음성을 멈추는 함수
def stop_speech():
    global is_speaking
    if is_speaking:
        speech_synthesizer.stop_speaking_async()
        is_speaking = False
    return "Speech stopped."

# 채팅
def chat_with_openai(user_input, chat_history):
    # 수어가 있는 경우
    try:
        global is_speaking
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()  # 속성 접근 방식으로 수정

        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        # 출처와 관련된 내용 확인
        citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
        video_url = citations[-2]
        image_urls = citations[-3].split(' ')  # 이미지 URL이 여러 개일 수 있음

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        is_speaking = True
        speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, '', video_url, image_urls, gr.update(visible=True), gr.update(visible=False)

    # 지문자로 대체하는 경우
    except Exception as e:
        # 오류 메시지 대체
        fallback_message = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"
        chat_history.append((user_input, fallback_message))

        return chat_history, '', None, [], gr.update(visible=False), gr.update(visible=True)

# Custom Vision
def process_frame(frame):
    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame = draw_boxes(frame, results.predictions)
        
        return annotated_frame
    
    except Exception as e:
        print(f"Error during prediction: {e}")
        return frame

def draw_boxes(image, predictions):
    """Draw bounding boxes on the image based on predictions"""
    img = image.copy()
    for pred in predictions:
        if pred.probability > 0.9 :
            color = (255, 0, 0)
            box = pred.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])
        
        # Draw rectangle
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
        
        # Add label with confidence score
            label = f"{pred.tag_name}: {pred.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
            stop_button = gr.Button("Stop Speech")
        with gr.Column(visible=False) as column1:   # 기본적으로 숨김
            video_display = gr.Video(label="수어 영상")
            image_display = gr.Gallery(label="수어 이미지", columns=3)
        with gr.Column(visible=False) as column2:   # 기본적으로 숨김
            with gr.Row():
                webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
                webcam_output = gr.Image(label="Detected Objects")
            show_btn = gr.Button("힌트")
    with Modal(visible=False) as modal:  # 지문자 힌트 보여주기
            gr.Textbox("이미지가 보여질 공간")


    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    webcam_input.stream(process_frame, inputs=webcam_input, outputs=webcam_output)
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, user_input, video_display, image_display, column1, column2])
    clear_button.click(
        lambda: ([], []),  # chatbot과 state를 모두 초기화
        inputs=None,
        outputs=[chatbot, state]  # chatbot과 state 모두 업데이트
    )
    stop_button.click(stop_speech)  # 음성을 멈추기
    show_btn.click(lambda: Modal(visible=True), None, modal)  # 지문자 힌트 보여주기
    
# 실행
demo.launch()

c:\python\Lib\site-packages\gradio\components\chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7886

To create a public link, set `share=True` in `launch()`.


c:\python\Lib\site-packages\gradio\blocks.py:1751: UserWarning: A function (stop_speech) returned too many output values (needed: 0, returned: 1). Ignoring extra values.
    Output components:
        []
    Output values returned:
        ["Speech stopped."]
  warnings.warn(
